<a href="https://colab.research.google.com/github/valentinaboriano/Computer-Vision/blob/main/fruit-inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
s

# Progetto CV